In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM


# `load_dataset` 함수로 CodeAlpaca-20k 데이터셋을 로드합니다.
# 이 데이터셋은 'Instruction'에 대한 'Output'을 제공하는 코드 생성 학습 데이터입니다.
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")
"""
- **Instruction**: 모델에게 주어지는 "질문" 또는 "가이드라인"입니다. 사용자가 원하는 코드나 기능을 설명하는 문장입니다.
- **Input(optional)**: 코드가 실행될 때 받는 입력입니다. 이 부분은 선택사항일 수 있으며, 필요한 경우에만 포함됩니다.
- **Output**: 사용자가 제시한 'Instruction'에 맞춰 모델이 생성해야 하는 'Answer'입니다. 이는 코드 출력이나 동작을 설명하는 부분입니다.
"""

# 모델과 토크나이저 로드
# 'facebook/opt-350m'은 OpenAI GPT 스타일의 언어 모델을 포함하는 사전 학습된 모델입니다.
# Causal Language Modeling을 위한 모델로, 텍스트 생성 작업에 적합합니다.
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")


# 'formatting_prompts_func'는 데이터셋 예시를 입력 받아, 'Instruction'과 'Output'을 적절한 형식으로 변환합니다.
# 각 'Instruction'과 'Output' 쌍을 연결하여 모델이 이를 처리할 수 있도록 합니다.
# 주어진 형식: '### Question: [Instruction]\n### Answer: [Output]'
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts


# 모델이 생성한 답변에 대한 템플릿을 정의합니다.
# 모델이 답변을 생성하는 부분을 표기합니다. '### Answer:'는 모델이 이 위치에서 답변을 시작할 것임을 나타냅니다.
response_template = " ### Answer:"

# 'DataCollatorForCompletionOnlyLM'은 데이터 로딩 시 모델의 요구에 맞게 데이터를 형성하는 데 사용됩니다.
# 주로 텍스트의 길이를 일정하게 맞추거나, 패딩을 추가하는 등의 역할을 합니다.
# 여기서 템플릿인 'response_template'을 사용하여 모델이 답변을 생성하는 위치를 지정합니다.
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


# SFTTrainer는 'Instruction-Tuning'을 위한 훈련 프로세스를 설정하는 클래스입니다.
# 이 클래스는 `SFTConfig`와 함께 사용되어 모델을 훈련하고 결과를 저장하는 데 필요한 설정을 포함합니다.
# 훈련을 위해 'train_dataset'을 사용하며, 'formatting_func'로 정의된 함수는 입력 데이터를 처리하고,
# 'data_collator'는 데이터 배치 처리 작업을 담당합니다.
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=SFTConfig(output_dir="/tmp/clm-instruction-tuning"),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

# `trainer.train()`을 호출하여 훈련을 시작합니다.
# 모델은 데이터셋을 사용하여 'Instruction-Tuning'을 학습하게 됩니다.
# 여기서 'Instruction-Tuning'은 모델이 주어진 'Instruction'에 맞춰 코드를 생성하는 능력을 향상시키는 훈련입니다.
trainer.train()

README.md:   0%|          | 0.00/677 [00:00<?, ?B/s]

(…)-00000-of-00001-e270777bb989ac86.parquet:   0%|          | 0.00/3.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

Step,Training Loss



KeyboardInterrupt

Process Process-auto_conversion:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/transformers/safetensors_conversion.py", line 101, in auto_conversion
    resolved_archive_file = cached_file(pretrained_model_name_or_path, filename, **cached_file_kwargs)
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/transformers/utils/hub.py", line 403, in cached_file
    resolved_file = hf_hub_download(
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
  File "/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/huggingface_hub/